In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/riyajolly/quesion-3/FT Data - data.csv
/kaggle/input/datasets/riyajolly/quesion-3/Unique Words Data - Sheet1.csv


In [32]:
df=pd.read_csv("/kaggle/input/datasets/riyajolly/quesion-3/FT Data - data.csv")
unique_df=pd.read_csv("/kaggle/input/datasets/riyajolly/quesion-3/Unique Words Data - Sheet1.csv")

In [33]:
#Fixing broken urls
df["trans_url_fixed"]=df["transcription_url_gcp"].str.replace("joshtalks-data-collection/hq_data/hi/", "upload_goai/", regex=False)
df["audio_url_fixed"]=df["rec_url_gcp"].str.replace("joshtalks-data-collection/hq_data/hi/", "upload_goai/", regex=False)

In [34]:
df.head()

,user_id,recording_id,language,duration,rec_url_gcp,transcription_url_gcp,metadata_url_gcp,trans_url_fixed,audio_url_fixed
0,245746,825780,hi,443,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/967...,https://storage.googleapis.com/upload_goai/967...
1,291038,825727,hi,443,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/967...,https://storage.googleapis.com/upload_goai/967...
2,246004,988596,hi,475,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/114...,https://storage.googleapis.com/upload_goai/114...
3,93626,990175,hi,475,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/114...,https://storage.googleapis.com/upload_goai/114...
4,286851,526266,hi,522,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/639...,https://storage.googleapis.com/upload_goai/639...


In [35]:
words=unique_df["word"].astype(str).str.strip()
words

0                       है
1                       तो
2                      में
3                       जी
4                      हैं
                ...       
177503    महत-महत्वपूर्णता
177504       कॉन्स्टिटूशनल
177505          विरण-विवरण
177506             दीदी...
177507            इंप्लॉइज़
Name: word, Length: 177508, dtype: object

In [36]:
words=words.str.replace(r'[^\u0900-\u097F]','',regex=True) #All unnecessymbols get removed
words=words[words!=""]
words

0                      है
1                      तो
2                     में
3                      जी
4                     हैं
               ...       
177503    महतमहत्वपूर्णता
177504      कॉन्स्टिटूशनल
177505          विरणविवरण
177506               दीदी
177507           इंप्लॉइज़
Name: word, Length: 176941, dtype: object

In [37]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.vec.gz
!gunzip cc.hi.300.vec.gz

--2026-02-21 16:30:10--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.hi.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.169.121.107, 3.169.121.81, 3.169.121.57, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.169.121.107|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1118942272 (1.0G) [binary/octet-stream]
Saving to: ‘cc.hi.300.vec.gz’

cc.hi.300.vec.gz    100%[===================>]   1.04G   153MB/s    in 7.1s    

2026-02-21 16:30:18 (150 MB/s) - ‘cc.hi.300.vec.gz’ saved [1118942272/1118942272]

gzip: cc.hi.300.vec already exists; do you wish to overwrite (y or n)? ^C


In [38]:
hindi_dict = set()

with open("cc.hi.300.vec", encoding="utf-8") as f:
    next(f)  # skip header
    for line in f:
        word = line.split()[0]
        hindi_dict.add(word)

In [39]:
#now using the Stemming approach to get a better result
suffixes = {
    1: [u"ो",u"े",u"ू",u"ु",u"ी",u"ि",u"ा",u"|"],
    2: [u"कर",u"ाओ",u"िए",u"ाई",u"ाए",u"ने",u"नी",u"ना",u"ते",u"ीं",u"ती",u"ता",u"ाँ",u"ां",u"ों",u"ें"],
    3: [u"ाकर",u"ाइए",u"ाईं",u"ाया",u"ेगी",u"ेगा",u"ोगी",u"ोगे",u"ाने",u"ाना",u"ाते",u"ाती",u"ाता",u"तीं",u"ाओं",u"ाएं",u"ुओं",u"ुएं",u"ुआं"],
    4: [u"ाएगी",u"ाएगा",u"ाओगी",u"ाओगे",u"एंगी",u"ेंगी",u"एंगे",u"ेंगे",u"ूंगी",u"ूंगा",u"ातीं",u"नाओं",u"नाएं",u"ताओं",u"ताएं",u"ियाँ",u"ियों",u"ियां"],
    5: [u"ाएंगी",u"ाएंगे",u"ाऊंगी",u"ाऊंगा",u"ाइयाँ",u"ाइयों",u"ाइयां"],
} #suffixes in hindi language that change tense, gender, number but not the meaning

In [40]:
import re
correct_words = []
incorrect_words = []
all_suffixes=[s for sublist in suffixes.values() for s in sublist]

for word in words.unique():
    if word.strip("।") in hindi_dict:
        correct_words.append(word)
        continue
    clean=re.sub(r'[^\u0900-\u097F]', '', word) #removing punctuations from the words
    clean=clean.strip("।")
    if clean=="":
        correct_words.append(word)
        continue
    if clean in hindi_dict:
        correct_words.append(word)
        continue
    if any(ch in clean for ch in ["ं", "ॉ", "ॅ", "्", "़"]):
        correct_words.append(word)
        continue
    is_correct=False
    if any(clean.endswith(s) for s in all_suffixes):
        for L in range(1,6): 
            if len(clean) >= L + 1:
                for suffix in suffixes[L]:
                    if clean.endswith(suffix):
                        pred=clean[:-len(suffix)]
                        if pred in hindi_dict:
                            correct_words.append(word)
                            is_correct=True
                            continue
    incorrect_words.append(word)

In [41]:
print("Total words:", len(words.unique()))
print("Correct words:", len(correct_words))
print("Incorrect words:", len(incorrect_words))

Total words: 162211
Correct words: 138707
Incorrect words: 30036


### The final results are as follows:
Total words: 162211
Correct words: 138707
Incorrect words: 30036

In [42]:
results = []

for w in correct_words:
    results.append([w, "correct spelling"])

for w in incorrect_words:
    results.append([w, "incorrect spelling"])

result_df = pd.DataFrame(results, columns=["word", "label"])

In [43]:
result_df.to_csv("final_spelling_labels.csv", index=False)

In [45]:
result_df

,word,label
0,है,correct spelling
1,तो,correct spelling
2,में,correct spelling
3,जी,correct spelling
4,हैं,correct spelling
...,...,...
168738,कमिटीज,incorrect spelling
168739,हुयूज,incorrect spelling
168740,हेवीचुअल,incorrect spelling
168741,सफरेन,incorrect spelling
